# torch.nn

In [1]:
import torch
import torch.nn as nn

## 0. important things about module ⭐️
- torch.nn.parameter.Parameter
- register_module_forward_pre_hook
- register_module_forward_hook
- register_module_backward_hook 

### torch.nn.parameter.Parameter
- torch.nn.parameter.Parameter(data, requires_grad=True)
- A kind of Tensor that is to be considered a module parameter
- Subclass(자식클래스) of Tensor
- 그냥 Tensor에 requires_grad=True하는 거랑 다른건가?

## 1. Containers

### Module ⭐️⭐️⭐️
- Base class for all neural network modules
- You can assign other submodules as regular attributes

#### Variables

##### training
- return bool whether this module is in training or evaluation mode

In [3]:
w = nn.Linear(2, 3)
w.training

True

In [4]:
import torchvision.models as models

alexnet = models.alexnet()

In [20]:
# 계층적으로 접근한다
for idx, module in enumerate(alexnet.modules()):
    print(f"{idx}번 째 모듈 {module}")

0번 째 모듈 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias

#### Methods

##### modules(), children(), named_modules(), named_children(), get_submodule(), add_module()
- 생각: 일단 이 메소드로 할 수 있어야 하는 것은 '모델 구조 확인', '임의의 모듈 접근', '임의의 모듈 수정/삭제', '원하는 위치에 커스텀 모듈 추가' 정도이다.

In [28]:
# 모델 구조 확인
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [92]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(
      inplace=True
      (100): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential

In [74]:
# 임의의 모듈 접근
# __dict__속성을 적극 활용하자
# alexnet.__dict__
# alexnet.features.__dict__

# alexnet에서 features에 바로 접근 가능한 이유가 뭐지..?
# 내 생각대로 라면 alexnet._modules["features"]._modules["0"] 이런식으로 접근해야 하는데
print(alexnet.features._modules["0"])
print(alexnet.avgpool)
print(alexnet.classifier._modules["6"])

# get_submodule()
print(alexnet.get_submodule("features.0"))
print(alexnet.get_submodule("avgpool"))

# 이게 최고인 것 같은데👈🤩
# Sequential 객체에 적힌 번호가 key값을 의미
print(alexnet.classifier[6])

Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
AdaptiveAvgPool2d(output_size=(6, 6))
Linear(in_features=4096, out_features=1000, bias=True)
Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
AdaptiveAvgPool2d(output_size=(6, 6))
Linear(in_features=4096, out_features=1000, bias=True)


In [106]:
# 이름을 모르겠을 때는 named_modules() 사용한다
# 계층적으로 접근한다
for name, module in alexnet.named_modules():
    print(f"이름: {name}")

이름: 
이름: features
이름: features.0
이름: features.1
이름: features.2
이름: features.3
이름: features.4
이름: features.5
이름: features.6
이름: features.7
이름: features.8
이름: features.9
이름: features.10
이름: features.11
이름: features.11.100
이름: features.12
이름: avgpool
이름: classifier
이름: classifier.0
이름: classifier.1
이름: classifier.2
이름: classifier.3
이름: classifier.4
이름: classifier.5
이름: classifier.6


In [85]:
# 임의의 모듈 수정/삭제

# 모듈 삭제
alexnet.classifier[6] = nn.Identity()

# 모듈 수정
alexnet.classifier[6] = nn.Linear(4096, 10)

In [ ]:
# 모듈 추가

# add_module()
# Adds a child👶 module to the current module
alexnet.features[11].add_module('100', nn.BatchNorm2d(256))

# child module이 아닌 그냥 module 추가하고 싶을 때는 어떻게 할까?

❓ child module이 아닌 그냥 module 추가하고 싶을 때는 어떻게 할까?

❓ 특정 모듈 뒤에 자식 모듈을 추가하는 것과 그냥 모듈을 추가하는 것에 어떤 차이가 있을까?

##### named_parameters(), parameters(), get_parameter()

In [93]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(
      inplace=True
      (100): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential

In [96]:
alexnet.features[0].parameters()

<generator object Module.parameters at 0x7fc8619b1bd0>

In [102]:
for param in alexnet.features[0].parameters():
    print(param.shape)

torch.Size([64, 3, 11, 11])
torch.Size([64])


In [103]:
# 특정 모듈 학습 안하도록 만들기(freeze)
for param in alexnet.features[0].parameters():
    param.requires_grad=False

In [105]:
for name, param in alexnet.features.named_parameters():
    print(f"파라미터 이름: {name}")

파라미터 이름: 0.weight
파라미터 이름: 0.bias
파라미터 이름: 3.weight
파라미터 이름: 3.bias
파라미터 이름: 6.weight
파라미터 이름: 6.bias
파라미터 이름: 8.weight
파라미터 이름: 8.bias
파라미터 이름: 10.weight
파라미터 이름: 10.bias
파라미터 이름: 11.100.weight
파라미터 이름: 11.100.bias


In [ ]:
# 파라미터 그래프에 찍어보려는 정도로 get_parameter() 쓸 수 있을 것 같다
alexnet.features.get_parameter("6.bias")

##### buffers(), get_buffer(), named_buffers()

##### cpu(), cuda()

##### to(), zero_grad(), requires_grad_()

##### train(), eval()

##### forward()

##### state_dict(), load_state_dict()

##### register_backward_hook(), register_buffer(), register_forward_hook(), register_forward_pre_hook(), regster_full_backward_hook(), register_parameter()

##### apply()

##### extra_repr()

##### share_memory()

##### type()

### Sequential
- 번호 key값으로 접근 가능하게 한다?

### ModuleList

### ModuleDict

### ParameterList

### ParameterDict

## 2. Convolution Layers

### nn.Conv1d

### nn.Conv2d

## 3. Pooling Layers

### nn.MaxPool1d, 2d

### nn.AvgPool1d, 2d

## 4. Padding Layers

## 5. Non-linear Activations ⭐️

### nn.LeakyReLU

### nn.ReLU

### nn.Sigmoid

### nn.Softmax

### nn.LogSoftmax

### nn.MultiheadAttention

## 6. Normalization Layers

### nn.BatchNorm1d, 2d

### nn.LayerNorm

## 7. Recurrent Layers

### nn.RNN

### nn.LSTM

### nn.GRU

## 8. Linear Layers

### nn.Identity

### nn.Linear

## 9. Dropout Layers

### nn.Dropout1d, 2d

## 10. Sparse Layers

### nn.Embedding

## 11. Loss Functions ⭐️

### nn. L1Loss

### nn.MSELoss

### nn.CrossEntropyLoss

## 12. Distance Functions

### nn.CosineSimilarity

## 13. Utilities

### nn.utils.clip_grand_norm_

### nn.utils.rnn.pack_padded_sequence

### nn.utils.rnn.pad_packd_sequence

### nn.Flatten